<div align="center">
    <h2>Υπολογιστική Γεωμετρία (Εργασία) - Γραμμικός Προγραμματισμός</h2>
</div>

Σε αυτό το notebook θα ασχοληθούμε με το δεύτερο μέρος της εργασίας το οποίο αφορά το **Γραμμικό Προγραμματισμό**. Συγκεκριμένα θα υλοποιήσουμε τον αυξητικό αλγόριθμο για την επίλυση ενός προβλήματος γραμμικού προγραμματισμού στο επίπεδο.

### Κατασκευή Αυξητικού Αλγορίθμου

In [6]:
import numpy as np

class LP2D_Restriction:
  def __init__(self, a_vector: np.ndarray[float], b: float):
    self.a_vector = a_vector
    self.b = b
    
class LP2D_ObjectiveFunction:
  def __init__(self, c_vector: np.ndarray[float]):
    self.c_vector = c_vector

In [7]:
class LP2D_SeidelAlgorithm:
  def __call__(self, A: np.ndarray[np.ndarray[float]], b: np.ndarray[float], c: np.ndarray[float], d: int):
    # Find the intersection points of the first d+1 constraints
    intersection_points = []
    if d == 2:
      # Find the optimized vertex of the feasible region that the first 3 constraints define
      constraints_a = A[0:d+3]
      constraints_b = b[0:d+3]
      
      for i in range(d+3):
        for j in range(i+1, d+3):
          point = np.linalg.solve(
            [constraints_a[i], constraints_a[j]], [constraints_b[i], constraints_b[j]]
          )
      
          # Make sure that the intersection points satisfy the constraints     
          if np.all(np.dot(constraints_a, point) <= constraints_b):
            intersection_points.append(point)      
    elif d == 1:
      constraints_a = A
      constraints_b = b
      for i in range(len(constraints_a)):    
        point = tuple([constraints_b[i]/constraints_a[i][0]])
        intersection_points.append(point)
          
    # Check if the feasible region is unbounded
    if not intersection_points:
      return None
    
    # Find the intersection point that maximizes the objective function
    optimized_point = min(intersection_points, key=lambda point: np.dot(c[:-1], point))
    
    # Use the remaining constraints to find the optimized point
    for i in range(d+3, len(A)):
      
      constraints_a = np.append(constraints_a, [A[i]], axis=0)
      constraints_b = np.append(constraints_b, b[i])
      
      # Check if the optimed point satisfies the new constraint
      if np.dot(constraints_a[-1], optimized_point) <= constraints_b[-1]:
        continue 
      else:
        
        # Solve the smaller problem to find the new optimized point
        new_constraints_a = np.empty((0, 1), float)
        new_constraints_b = np.empty((0,), float)
        for j in range(i):
          new_row = np.array([[constraints_a[j][0] - constraints_a[j][1] * (constraints_a[-1][0] / constraints_a[-1][1])]])
          new_constraints_a = np.vstack([new_constraints_a, new_row])
          new_constraints_b = np.append(new_constraints_b, constraints_b[j] - constraints_a[j][1] * (constraints_b[-1] / constraints_a[-1][1]))
          
        new_c = []
        new_c.append(c[0] - c[1]*(constraints_a[-1][0]/constraints_a[-1][1]))
        new_c.append(c[1]*(constraints_b[-1]/constraints_a[-1][1]) + c[2])
        
        new_optimized_point = self(new_constraints_a, new_constraints_b, new_c, d-1)
        optimized_point = tuple(
          np.append(
            new_optimized_point,
            ((constraints_a[-1][0] / (-constraints_a[-1][1]))*new_optimized_point[0]) + ((-constraints_b[-1]) / (-constraints_a[-1][1]))
          )
        )
       
    return optimized_point

Ας δοκιμάσουμε τον παραπάνω αλγόριθμο για μία **αντικειμενική συνάρτηση** (objective function) και **ορισμένους περιορισμούς** (constraints). Συγκεκριμένα τα δεδομένα που θα χρησιμοποιήσουμε θα είναι τα εξής:
1. $\max\{3x_1 - 10x_2\}$
2. υπό τους περιορισμούς:
    - $-2x_1 + x_2 \leq 12$
    - $x_1 -3x_2 \geq -3$
    - $6x_1 + 7x_2 \leq 18$
    - $-3x_1 + 12x_2 \geq 8$
    - $2x_1 - 7x_2 \leq 35$
    - $-x_1 + 8x_2 \leq 29$
    - $-2x_1 + 6x_2 \geq -9$
    - $x_1,x_2 \geq 0$
    
Για να μπορέσουμε να λύσουμε το παραπάνω πρόβλημα γραμμικού προγραμματισμού, θα πρέπει να μετατρέψουμε τα δεδομένα μας σε κατάλληλη μορφή που επιτρέπει ο αλγόριθμος. Συγκεκριμένα θα πρέπει το πρόβλημά μας να έχει τη μορφή:

$$
  \min\{f(x)\},\ and\ H_i(x) \leq 0\ for\ \ i=1,2,\dots,n
$$

όπου $H_i(x)$ είναι οι $n$ περιορισμοί του προβλήματος και $f(x)$ η αντικειμενική συνάρτηση. Με βάση τη προηγούμενη μορφή το πρόβλημα που έχουμε να λύσουμε μετατρέπεται σε:
1. $\min\{-3x_1 + 10x_2\}$
2. υπό τους περιορισμούς:
    - $-2x_1 + x_2 -12 \leq 0$
    - $-x_1 + 3x_2 -3 \leq 0$
    - $6x_1 + 7x_2 - 18 \leq 0$
    - $3x_1 - 12x_2 + 8\leq 0$
    - $2x_1 - 7x_2 -35 \leq 0$
    - $-x_1 + 8x_2 -29 \leq 0$
    - $2x_1 - 6x_2 -9 \leq 0$
    - $-x_1 \leq 0$
    - $-x_2 \leq 0$
    
Έχοντας μετατρέψει το πρόβλημά μας σε κατάλληλη μορφή για τον αλγόριθμο, ήρθε η ώρα να το λύσουμε. Ας περάσουμε τα δεδομένα μας στην Python.

In [8]:
A = np.array([
  [-1, 0],
  [0, -1],
  [-2, 1],
  [-1, 3],
  [6, 7],
  [3, -12],
  [2, -7],
  [-1, 8],
  [2, -6]
])

b = np.array([0, 0, 12, 3, 18, -8, 35, 29, 9])
c = np.array([-3, 10, 0])

και ας λύσουμε το πρόβλημα.

In [9]:
lp2d_algorithm = LP2D_SeidelAlgorithm()
optimized_point = lp2d_algorithm(A, b, c, 2)
optimized_value = -(c[0]*optimized_point[0] + c[1]*optimized_point[1] + c[2])

if optimized_point:
  print(f"The optimized point is ({optimized_point[0]}, {optimized_point[1]}) and the optimized value is {optimized_value}")
else:
  print("The feasible region is unbounded")

The optimized point is (1.7204301075268817, 1.096774193548387) and the optimized value is -5.806451612903225


Όπως παρατηρούμε η λύση του παραπάνω προβλήματος γραμμικού προγραμματισμού είναι το σημείο $(1.72, 1.09)$ και η τιμή της αντικειμενικής συνάρτησης σε αυτό το σημείο είναι $-5.80$

### Χρήση έτοιμης βιβλιοθήκης
Τέλος θα χρησιμοποιήσουμε μία έτοιμη βιβλιοθήκη για την επίλυση του παραπάνω προβλήματος, έτσι ώστε να επαληθεύσουμε την ορθότητα της λύσης μας. Συγκεκριμένα θα χρησιμοποιήσουμε το **pulp**.

In [10]:
# import the library pulp as p 
import pulp as p 

# Create a LP Minimization problem 
Lp_prob = p.LpProblem('Problem', p.LpMinimize) 

# Create problem Variables 
x1 = p.LpVariable("x1", lowBound = 0) # Create a variable x >= 0 
x2 = p.LpVariable("x2", lowBound = 0) # Create a variable y >= 0 

# Objective Function 
Lp_prob += -3*x1 + 10*x2 

# Constraints:
Lp_prob += -2*x1 + x2 <= 12
Lp_prob += x1 - 3*x2 >= -3
Lp_prob += 6*x1 + 7*x2 <= 18
Lp_prob += -3*x1 + 12*x2 >= 8
Lp_prob += 2*x1 - 7*x2 <= 35
Lp_prob += -x1 + 8*x2 <= 29
Lp_prob += -2*x1 + 6*x2 >= -9


status = Lp_prob.solve() # Solver 
print(p.LpStatus[status]) # The solution status 

# Printing the final solution 
print(p.value(x1), p.value(x2), -p.value(Lp_prob.objective)) 


Optimal
1.7204301 1.0967742 -5.806451700000001


Όπως παρατηρούμε η λύση που επέφερε το pulp είναι ίδια με αυτή που βρήκαμε προηγουμένως.